In [26]:
import os
import time
import boto3
import logging
import pandas as pd
from typing import Dict
from io import StringIO

# Logging setup
logging.basicConfig(
    format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Athena + S3 configuration (update with your details)
AWS_REGION = "us-east-2"       
DATABASE_NAME = "eth"           # from Glue (Ethereum database)
WORKGROUP = "primary"           # Athena workgroup name
S3_BUCKET_NAME = "athena-erika"  
S3_STAGING_PREFIX = "data/results"
S3_OUTPUT = f"s3://{S3_BUCKET_NAME}/{S3_STAGING_PREFIX}/"

print(f"✅ Results will be written to: {S3_OUTPUT}")

✅ Results will be written to: s3://athena-erika/data/results/


In [27]:
athena_client = boto3.client("athena", region_name=AWS_REGION)
s3_client = boto3.client("s3", region_name=AWS_REGION)

In [28]:
# To check that the S3 bucket exists
try:
    s3_client.head_bucket(Bucket=S3_BUCKET_NAME)
    print(f"✅ Bucket {S3_BUCKET_NAME} exists and is accessible.")
except Exception as e:
    print(f"❌ Error accessing bucket: {e}")

✅ Bucket athena-erika exists and is accessible.


In [29]:
# def download_and_load_query_results(client, query_response, download=True):
#     logger.info("⏳ Waiting for query to finish...")
#     query_id = query_response["QueryExecutionId"]

#     # Wait for query to complete
#     while True:
#         status = client.get_query_execution(QueryExecutionId=query_id)
#         state = status["QueryExecution"]["Status"]["State"]
#         if state in ["SUCCEEDED", "FAILED", "CANCELLED"]:
#             break
#         time.sleep(0.5)

#     if state != "SUCCEEDED":
#         raise Exception(f"Query failed: {state}")

#     # ✅ Preview mode: just show query metadata
#     if not download:
#         query_info = status["QueryExecution"]
#         stats = query_info["Statistics"]
#         logger.info(f"✅ Query succeeded. Data scanned: {stats['DataScannedInBytes']/1e6:.2f} MB")
#         logger.info(f"Output location: {query_info['ResultConfiguration']['OutputLocation']}")
#         return None

#     # Otherwise download and load into pandas
#     temp_file = f"athena_result_{query_id}.csv"
#     s3_key = f"{S3_STAGING_PREFIX}/{query_id}.csv"

#     s3_client.download_file(S3_BUCKET_NAME, s3_key, temp_file)
#     df = pd.read_csv(temp_file)
#     logger.info(f"✅ Query completed. Rows: {len(df)}")
#     return df

def download_and_load_query_results(client, query_response, filename="athena_result", download=True):
    logger.info("⏳ Waiting for query to finish...")
    query_id = query_response["QueryExecutionId"]

    # Wait until query completes
    while True:
        status = client.get_query_execution(QueryExecutionId=query_id)
        state = status["QueryExecution"]["Status"]["State"]
        if state in ["SUCCEEDED", "FAILED", "CANCELLED"]:
            break
        time.sleep(0.5)

    if state != "SUCCEEDED":
        raise Exception(f"Query failed: {state}")

    # Preview-only mode
    if not download:
        query_info = status["QueryExecution"]
        stats = query_info["Statistics"]
        logger.info(f"✅ Query succeeded. Data scanned: {stats['DataScannedInBytes']/1e6:.2f} MB")
        logger.info(f"Output location: {query_info['ResultConfiguration']['OutputLocation']}")
        return None

    # ✅ Step 1: Read result from S3
    s3_key = f"{S3_STAGING_PREFIX}/{query_id}.csv"
    obj = s3_client.get_object(Bucket=S3_BUCKET_NAME, Key=s3_key)
    df = pd.read_csv(StringIO(obj["Body"].read().decode("utf-8")))

    # ✅ Step 2: Save with a friendly name
    local_filename = f"{filename}.csv"
    df.to_csv(local_filename, index=False)
    logger.info(f"📁 Saved query result locally as {local_filename}")

    # (Optional) Step 3: Re-upload with same nice name to S3
    s3_client.upload_file(local_filename, S3_BUCKET_NAME, f"{S3_STAGING_PREFIX}/{local_filename}")
    logger.info(f"Uploaded to s3://{S3_BUCKET_NAME}/{S3_STAGING_PREFIX}/{local_filename}")

    logger.info(f"✅ Query completed. Rows: {len(df)}")
    return df


# DATA ANALYSIS
## 1. Overview of Ethereum Blocks

In [33]:
query = '''
SELECT 
    MIN(date) AS earliest_date,
    MAX(date) AS latest_date,
    COUNT(DISTINCT date) AS days_of_data,
    SUM(transaction_count) AS total_transactions,
    COUNT(*) AS total_blocks
FROM eth.blocks;
'''

start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={"OutputLocation": S3_OUTPUT},
)

# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------
download_and_load_query_results(athena_client, response, download=False)

# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()
if confirm == "y":
    network_overview = download_and_load_query_results(athena_client, response,  filename="network_overview", download=True)
    display(network_overview.head())

logger.info(f"Data fetched in {time.time() - start_time:.2f}s")


[2025-10-10 20:14:27,926] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:14:35,759] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 31.08 MB
[2025-10-10 20:14:35,759] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/441b1c67-924c-487b-9ebb-688e8d050390.csv
[2025-10-10 20:14:37,575] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:14:37,826] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as network_overview.csv
[2025-10-10 20:14:37,924] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/network_overview.csv
[2025-10-10 20:14:37,925] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 1


,earliest_date,latest_date,days_of_data,total_transactions,total_blocks
0,2015-07-30,2025-10-09,3725,3032681982,23543615


[2025-10-10 20:14:37,930] p21995 {4184522636.py:31} INFO - Data fetched in 10.13s


## 2. Network Health & Activity Trends

### How has daily network activity evolved over the past 30 days?

In [34]:
query = '''
SELECT 
    date,
    COUNT(DISTINCT number) as total_blocks,
    SUM(transaction_count) as total_transactions,
    CAST(AVG(gas_used) AS BIGINT) as avg_gas_used,
    ROUND(AVG(CAST(gas_used AS DOUBLE) / CAST(gas_limit AS DOUBLE) * 100), 4) as avg_block_utilization_pct,
    SUM(size) / 1024 / 1024 as total_data_mb
FROM eth.blocks
WHERE date >= DATE_FORMAT(CURRENT_DATE - INTERVAL '30' DAY, '%Y-%m-%d')
GROUP BY date
ORDER BY date DESC;
'''

start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------
download_and_load_query_results(athena_client, response, download=False)

# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()
if confirm == "y":
    daily_network_activity_last30days = download_and_load_query_results(athena_client, response,  filename="daily_network_activity_last30days", download=True)
    display(daily_network_activity_last30days.head())

logger.info(f"Data fetched in {time.time() - start_time:.2f}s")


[2025-10-10 20:14:54,081] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:14:55,821] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 3.50 MB
[2025-10-10 20:14:55,823] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/04b67cce-7d7b-4320-af16-30620eb5a8b7.csv
[2025-10-10 20:14:57,935] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:14:58,054] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as daily_network_activity_last30days.csv
[2025-10-10 20:14:58,152] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/daily_network_activity_last30days.csv
[2025-10-10 20:14:58,153] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 29


,date,total_blocks,total_transactions,avg_gas_used,avg_block_utilization_pct,total_data_mb
0,2025-10-09,7155,1594069,22722948,50.5099,851
1,2025-10-08,7140,1616085,22716619,50.4993,858
2,2025-10-07,7158,1595109,22725828,50.5119,876
3,2025-10-06,7151,1576776,22768085,50.6119,842
4,2025-10-05,7163,1387207,22773428,50.6237,785


[2025-10-10 20:14:58,161] p21995 {717619233.py:38} INFO - Data fetched in 4.18s


## 3. Gas Economics & Fee Analysis 

### What are the gas price patterns and transaction fee economics in the last 7 days?

In [37]:
query = '''
-- Gas Price Analysis & Fee Distribution (last 7 days)
SELECT 
    date,
    COUNT(*) as tx_count,
    AVG(CAST(gas_price AS DOUBLE) / 1e9) as avg_gas_price_gwei,
    APPROX_PERCENTILE(CAST(gas_price AS DOUBLE) / 1e9, 0.5) as median_gas_price_gwei,
    APPROX_PERCENTILE(CAST(gas_price AS DOUBLE) / 1e9, 0.95) as p95_gas_price_gwei,
    MAX(CAST(gas_price AS DOUBLE) / 1e9) as max_gas_price_gwei,
    SUM(CAST(receipt_gas_used AS DOUBLE) * CAST(gas_price AS DOUBLE)) / 1e18 as total_fees_eth,
    AVG(CAST(receipt_gas_used AS DOUBLE)) as avg_gas_per_tx
FROM eth.transactions
WHERE date >= DATE_FORMAT(CURRENT_DATE - INTERVAL '7' DAY, '%Y-%m-%d')
    AND receipt_status = 1
GROUP BY date
ORDER BY date DESC;
'''
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)           
# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------
download_and_load_query_results(athena_client, response, download=False)  


          

# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()
if confirm == "y":
    gas_price_analysis_last7days = download_and_load_query_results(athena_client, response,  filename="gas_price_analysis_last7days", download=True)
    display(gas_price_analysis_last7days.head())

logger.info(f"Data fetched in {time.time() - start_time:.2f}s")

[2025-10-10 20:19:07,475] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:19:09,732] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 72.46 MB
[2025-10-10 20:19:09,733] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/0a060568-9a8e-48f4-9b57-30060d74707a.csv
[2025-10-10 20:19:12,422] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:19:12,680] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as gas_price_analysis_last7days.csv
[2025-10-10 20:19:12,801] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/gas_price_analysis_last7days.csv
[2025-10-10 20:19:12,802] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 6


,date,tx_count,avg_gas_price_gwei,median_gas_price_gwei,p95_gas_price_gwei,max_gas_price_gwei,total_fees_eth,avg_gas_per_tx
0,2025-10-09,1575935,1.440535,0.828440,3.171930,44031.574685,211.525439,101199.616396
1,2025-10-08,1599094,1.181980,0.619738,2.667649,10631.669921,175.182301,99792.565507
2,2025-10-07,1574221,1.739233,0.974871,4.835812,8871.049230,265.913742,100896.875739
3,2025-10-06,1551318,1.811592,1.117077,4.814108,22259.295669,274.381513,102687.226762
4,2025-10-05,1369725,0.978103,0.281156,2.176368,41202.812041,130.959039,117250.405545


[2025-10-10 20:19:12,812] p21995 {563769666.py:43} INFO - Data fetched in 5.42s


## 4. Whale Wallet Activity

### Who are the biggest ETH movers and what are they doing?

In [39]:
query = '''
--- Top 20 Addresses by Transaction Volume (Last 7 Days)
SELECT 
    from_address,
    COUNT(*) AS tx_count,
    SUM(CAST(value AS DOUBLE)) / 1e18 AS total_eth_sent,
    AVG(CAST(value AS DOUBLE)) / 1e18 AS avg_eth_per_tx,
    SUM(CAST(receipt_gas_used AS DOUBLE) * CAST(gas_price AS DOUBLE)) / 1e18 AS total_gas_paid_eth,
    COUNT(CASE WHEN to_address IS NULL THEN 1 END) AS contracts_deployed,
    COUNT(DISTINCT to_address) AS unique_recipients
FROM eth.transactions
WHERE date >= DATE_FORMAT(CURRENT_DATE - INTERVAL '7' DAY, '%Y-%m-%d')
  AND value > 0
GROUP BY from_address
HAVING SUM(CAST(value AS DOUBLE)) / 1e18 > 100  -- Moved more than 100 ETH
ORDER BY total_eth_sent DESC
LIMIT 20;

'''
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)           
# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------
download_and_load_query_results(athena_client, response, download=False)            

# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()
if confirm == "y":
    top_whale_wallets_last7days = download_and_load_query_results(athena_client, response,  filename="top_whale_wallets_last7days", download=True)
    display(top_whale_wallets_last7days.head())

logger.info(f"Data fetched in {time.time() - start_time:.2f}s")


[2025-10-10 20:23:37,962] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:23:42,993] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 634.77 MB
[2025-10-10 20:23:42,994] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/e896fca9-4d52-44bd-b06c-1fd226cfe39b.csv
[2025-10-10 20:23:47,349] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:23:47,744] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as top_whale_wallets_last7days.csv
[2025-10-10 20:23:47,861] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/top_whale_wallets_last7days.csv
[2025-10-10 20:23:47,861] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 20


,from_address,tx_count,total_eth_sent,avg_eth_per_tx,total_gas_paid_eth,contracts_deployed,unique_recipients
0,0x28c6c06298d514db089934071355e5743bf21d60,12436,812185.410367,65.309216,0.203612,0,10540
1,0xb5d85cbf7cb3ee0d56b3bb207d5fc4b82f43f511,1738,491387.840264,282.731784,0.053059,0,1650
2,0xcd531ae9efcce479654c4926dec5f6209531ca7b,182,341981.188096,1879.017517,0.003597,0,122
3,0xceb69f6342ece283b2f5c9088ff249b5d0ae66ea,169,336212.856277,1989.425185,0.003990,0,109
4,0xeae7380dd4cef6fbd1144f49e4d1e6964258a4f4,238,288677.541128,1212.930845,0.003463,0,1


[2025-10-10 20:23:47,879] p21995 {3057630092.py:42} INFO - Data fetched in 10.23s


## 5. Smart Contract Deployment Trends

### What's the rate of smart contract creation for all transactions, and deployed?

In [40]:
Query = '''
-- Smart Contract Deployment Trends (Last 30 Days)
SELECT 
    date,
    COUNT(*) as contracts_deployed,
    COUNT(DISTINCT from_address) as unique_deployers,
    AVG(CAST(receipt_gas_used AS DOUBLE)) as avg_deployment_gas,
    SUM(CAST(receipt_gas_used AS DOUBLE) * CAST(gas_price AS DOUBLE)) / 1e18 as total_deployment_cost_eth
FROM eth.transactions
WHERE date >= DATE_FORMAT(CURRENT_DATE - INTERVAL '30' DAY, '%Y-%m-%d')
    AND receipt_status = 1
GROUP BY date
ORDER BY date DESC;
'''
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------
download_and_load_query_results(athena_client, response, download=False)

# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()
if confirm == "y":
    contract_deployment_trends_last30days = download_and_load_query_results(athena_client, response,  filename="contract_deployment_trends_last30days", download=True)
    display(contract_deployment_trends_last30days.head())

logger.info(f"Data fetched in {time.time() - start_time:.2f}s")

[2025-10-10 20:25:50,117] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:25:55,170] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 634.77 MB
[2025-10-10 20:25:55,170] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/d0e88d82-1a10-4941-9acc-eee3dc85f679.csv
[2025-10-10 20:25:58,311] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:25:58,580] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as contract_deployment_trends_last30days.csv
[2025-10-10 20:25:58,687] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/contract_deployment_trends_last30days.csv
[2025-10-10 20:25:58,688] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 20


,from_address,tx_count,total_eth_sent,avg_eth_per_tx,total_gas_paid_eth,contracts_deployed,unique_recipients
0,0x28c6c06298d514db089934071355e5743bf21d60,12436,812185.410367,65.309216,0.203612,0,10540
1,0xb5d85cbf7cb3ee0d56b3bb207d5fc4b82f43f511,1738,491387.840264,282.731784,0.053059,0,1650
2,0xcd531ae9efcce479654c4926dec5f6209531ca7b,182,341981.188096,1879.017517,0.003597,0,122
3,0xceb69f6342ece283b2f5c9088ff249b5d0ae66ea,169,336212.856277,1989.425185,0.003990,0,109
4,0xeae7380dd4cef6fbd1144f49e4d1e6964258a4f4,238,288677.541128,1212.930845,0.003463,0,1


[2025-10-10 20:25:58,698] p21995 {564524190.py:38} INFO - Data fetched in 8.91s


### What are the top contract deployers?

In [41]:
query = '''
SELECT 
    from_address as deployer,
    COUNT(*) as contracts_deployed,
    AVG(CAST(receipt_gas_used AS DOUBLE)) as avg_gas_used,
    SUM(CAST(receipt_gas_used AS DOUBLE) * CAST(gas_price AS DOUBLE)) / 1e18 as total_spent_eth
FROM eth.transactions
WHERE date >= DATE_FORMAT(CURRENT_DATE - INTERVAL '30' DAY, '%Y-%m-%d')
    AND receipt_status = 1
GROUP BY from_address
ORDER BY contracts_deployed DESC
LIMIT 10;

'''
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------
download_and_load_query_results(athena_client, response, download=False)

# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()
if confirm == "y":
    top_contract_deployers_last30days = download_and_load_query_results(athena_client, response,  filename="top_contract_deployers_last30days", download=True)
    display(top_contract_deployers_last30days.head())

logger.info(f"Data fetched in {time.time() - start_time:.2f}s")

[2025-10-10 20:28:10,864] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:28:15,343] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 1955.54 MB
[2025-10-10 20:28:15,345] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/bbfcb49a-0c34-4f85-bee2-1c0ddfbc2cae.csv
[2025-10-10 20:28:17,973] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:28:18,230] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as top_contract_deployers_last30days.csv
[2025-10-10 20:28:18,324] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/top_contract_deployers_last30days.csv
[2025-10-10 20:28:18,325] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 10


,deployer,contracts_deployed,avg_gas_used,total_spent_eth
0,0xdadb0d80178819f2319190d340ce9a924f783711,488736,22843.223677,5.790166
1,0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97,405121,21570.371669,4.972262
2,0xf70da97812cb96acdf810712aa562db8dfa3dbef,405113,99463.609252,50.077945
3,0x28c6c06298d514db089934071355e5743bf21d60,369453,68458.736830,33.699671
4,0x559432e18b281731c054cd703d4b49872be4ed53,324497,53619.622262,23.965474


[2025-10-10 20:28:18,331] p21995 {2372339901.py:37} INFO - Data fetched in 7.69s


## 6. Token Transfer Activity (ERC-20 Analysis)

### Which tokens are most actively traded?

In [43]:
query = '''
-- Token Transfer Activity (Last 7 Days)
SELECT 
    token_address,
    COUNT(*) as transfer_count,
    COUNT(DISTINCT from_address) as unique_senders,
    COUNT(DISTINCT to_address) as unique_receivers,
    SUM(CAST(value AS DOUBLE)) / 1e18  as total_volume,
    AVG(CAST(value AS DOUBLE)) / 1e18  as avg_transfer_size
FROM eth.token_transfers
WHERE date >= DATE_FORMAT(CURRENT_DATE - INTERVAL '7' DAY, '%Y-%m-%d')
GROUP BY token_address
ORDER BY transfer_count DESC
LIMIT 20;
'''
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------
download_and_load_query_results(athena_client, response, download=False)
# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()
if confirm == "y":
    top_token_transfers_last7days = download_and_load_query_results(athena_client, response,  filename="top_token_transfers_last7days", download=True)  
    display(top_token_transfers_last7days.head())
logger.info(f"Data fetched in {time.time() - start_time:.2f}s")

[2025-10-10 20:33:42,649] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:33:46,597] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 1016.53 MB
[2025-10-10 20:33:46,598] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/9904711b-d99b-4800-afbc-53ed70ee1f3f.csv
[2025-10-10 20:33:48,626] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:33:48,888] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as top_token_transfers_last7days.csv
[2025-10-10 20:33:48,999] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/top_token_transfers_last7days.csv
[2025-10-10 20:33:49,000] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 20


,token_address,transfer_count,unique_senders,unique_receivers,total_volume,avg_transfer_size
0,0xdac17f958d2ee523a2206206994597c13d831ec7,2558147,504434,647437,1.187738e-01,4.642963e-08
1,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2314608,364356,451017,2.406890e-01,1.039869e-07
2,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1663751,38055,35330,1.028764e+08,6.183403e+01
3,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,647372,15924,619594,2.437544e+13,3.765290e+07
4,0x06450dee7fd2fb8e39061434babcfc05599a6fb8,290301,906,143448,8.868567e+13,3.054956e+08


[2025-10-10 20:33:49,010] p21995 {778610392.py:36} INFO - Data fetched in 6.76s


## 7. Transaction Success Rate & Failed Transactions

### What's the failure rate and why are transactions failing?

In [44]:
query = '''
-- Transaction Success Analysis (Last 30 Days)
SELECT 
    date,
    COUNT(*) as total_tx,
    SUM(CASE WHEN receipt_status = 1 THEN 1 ELSE 0 END) as successful_tx,
    SUM(CASE WHEN receipt_status = 0 THEN 1 ELSE 0 END) as failed_tx,
    CAST(SUM(CASE WHEN receipt_status = 0 THEN 1 ELSE 0 END) AS DOUBLE) / COUNT(*) * 100 as failure_rate_pct,
    -- Gas wasted on failed transactions
    SUM(CASE WHEN receipt_status = 0 
        THEN CAST(receipt_gas_used AS DOUBLE) * CAST(gas_price AS DOUBLE) 
        ELSE 0 END) / 1e18 as gas_wasted_eth
FROM eth.transactions
WHERE date >= DATE_FORMAT(CURRENT_DATE - INTERVAL '30' DAY, '%Y-%m-%d')
GROUP BY date
ORDER BY date DESC;
'''

start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------
download_and_load_query_results(athena_client, response, download=False)

# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()
if confirm == "y":
    tx_success_analysis_last30days = download_and_load_query_results(athena_client, response,  filename="tx_success_analysis_last30days", download=True)  
    display(tx_success_analysis_last30days.head())
logger.info(f"Data fetched in {time.time() - start_time:.2f}s")

[2025-10-10 20:35:47,045] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:35:50,496] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 376.74 MB
[2025-10-10 20:35:50,497] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/35d217ec-8a87-4f7d-bfe2-1f4b56d2415b.csv
[2025-10-10 20:35:57,888] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:35:58,147] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as tx_success_analysis_last30days.csv
[2025-10-10 20:35:58,263] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/tx_success_analysis_last30days.csv
[2025-10-10 20:35:58,264] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 29


,date,total_tx,successful_tx,failed_tx,failure_rate_pct,gas_wasted_eth
0,2025-10-09,1594069,1575935,18134,1.137592,3.086095
1,2025-10-08,1616085,1599094,16991,1.051368,2.374936
2,2025-10-07,1595109,1574221,20888,1.309503,4.735454
3,2025-10-06,1576776,1551318,25458,1.614560,7.222960
4,2025-10-05,1387207,1369725,17482,1.260230,3.034394


[2025-10-10 20:35:58,270] p21995 {748015229.py:40} INFO - Data fetched in 11.49s


### What are high-value failed transactions in the last 7 days? 

In [45]:
query = '''
-- Expensive Failed Transactions (Last 7 Days)
SELECT 
    hash,
    from_address,
    to_address,
    CAST(value AS DOUBLE) / 1e18 as eth_amount,
    CAST(receipt_gas_used AS DOUBLE) * CAST(gas_price AS DOUBLE) / 1e18 as gas_wasted_eth,
    block_number,
    block_timestamp
FROM eth.transactions
WHERE date >= DATE_FORMAT(CURRENT_DATE - INTERVAL '7' DAY, '%Y-%m-%d')
    AND receipt_status = 0
    AND CAST(receipt_gas_used AS DOUBLE) * CAST(gas_price AS DOUBLE) / 1e18 > 0.1 -- Lost more than 0.1 ETH in gas
ORDER BY gas_wasted_eth DESC
LIMIT 20;
'''
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------

download_and_load_query_results(athena_client, response, download=False)

# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()
if confirm == "y":
    expensive_failed_txs_last7days = download_and_load_query_results(athena_client, response,  filename="expensive_failed_txs_last7days", download=True)  
    display(expensive_failed_txs_last7days.head())
logger.info(f"Data fetched in {time.time() - start_time:.2f}s")

[2025-10-10 20:38:27,923] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:38:30,731] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 113.93 MB
[2025-10-10 20:38:30,731] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/e093985e-22d2-437e-939f-cf67c070d829.csv
[2025-10-10 20:38:35,568] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:38:35,984] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as expensive_failed_txs_last7days.csv
[2025-10-10 20:38:36,077] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/expensive_failed_txs_last7days.csv
[2025-10-10 20:38:36,077] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 3


,hash,from_address,to_address,eth_amount,gas_wasted_eth,block_number,block_timestamp
0,0xab199e9a227093fff7595045602f80824ebea27250d1...,0xcdc89e8546e2f4f425de825ba152d6a0438fb25b,0x38778e6d4d0dbe9becef3ae8b938570209efa48b,1.418719,0.255856,23513433,2025-10-05 18:43:23.000
1,0xe21a914d96d862fcccd40c6e40b3555eb6c3ab76d68d...,0xcdc89e8546e2f4f425de825ba152d6a0438fb25b,0x38778e6d4d0dbe9becef3ae8b938570209efa48b,1.417911,0.255856,23513431,2025-10-05 18:42:59.000
2,0x40fa1815532e4eb47eabf15a41c0a63ac6809e14dba7...,0xcdc89e8546e2f4f425de825ba152d6a0438fb25b,0x38778e6d4d0dbe9becef3ae8b938570209efa48b,1.415832,0.109652,23513429,2025-10-05 18:42:35.000


[2025-10-10 20:38:36,084] p21995 {3166031474.py:40} INFO - Data fetched in 8.45s


## 8. Mining/Validator Analysis

### Who are the dominant block producers and what are they earning?


In [46]:
query = '''
-- Top Block Producers (Validators/Miners)
SELECT 
    miner,
    COUNT(*) as blocks_produced,
    SUM(transaction_count) as total_tx_processed,
    AVG(CAST(gas_used AS DOUBLE)) as avg_gas_per_block,
    -- Estimate block rewards (simplified - doesn't include all MEV)
    COUNT(*) * 2 as estimated_base_rewards_eth, -- ~2 ETH per block post-merge
    SUM(CAST(base_fee_per_gas AS DOUBLE) * CAST(gas_used AS DOUBLE)) / 1e18 as base_fees_burned_eth
FROM eth.blocks
WHERE date >= DATE_FORMAT(CURRENT_DATE - INTERVAL '7' DAY, '%Y-%m-%d')
GROUP BY miner
ORDER BY blocks_produced DESC
LIMIT 15;
'''
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------
download_and_load_query_results(athena_client, response, download=False)        

# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()        
if confirm == "y":
    top_block_producers_last7days = download_and_load_query_results(athena_client, response,  filename="top_block_producers_last7days", download=True)  
    display(top_block_producers_last7days.head())
logger.info(f"Data fetched in {time.time() - start_time:.2f}s")

[2025-10-10 20:40:37,682] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:40:39,385] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 0.64 MB
[2025-10-10 20:40:39,386] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/06d0f261-6cae-439f-bcd1-810fc725798a.csv
[2025-10-10 20:40:41,008] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:40:41,248] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as top_block_producers_last7days.csv
[2025-10-10 20:40:41,336] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/top_block_producers_last7days.csv
[2025-10-10 20:40:41,336] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 15


,miner,blocks_produced,total_tx_processed,avg_gas_per_block,estimated_base_rewards_eth,base_fees_burned_eth
0,0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97,20282,4596752,2.464047e+07,40564,216.128811
1,0xdadb0d80178819f2319190d340ce9a924f783711,12596,2958879,2.551746e+07,25192,126.699329
2,0x396343362be2a4da1ce0c1c210945346fb82aa49,2794,529615,1.958853e+07,5588,23.187007
3,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,1557,350816,2.275277e+07,3114,17.417579
4,0x1f9090aae28b8a3dceadf281b0f12828e676c326,1431,207165,1.409060e+07,2862,4.010539


[2025-10-10 20:40:41,342] p21995 {3997627251.py:38} INFO - Data fetched in 3.97s


## 9. Network Congestion & Peak Usage Patterns

### When is the network most congested? Identify peak usage times.

In [47]:
query = '''
SELECT 
    HOUR(block_timestamp) AS hour_of_day,
    COUNT(DISTINCT b.number) AS total_blocks,
    SUM(b.transaction_count) AS total_transactions,
    AVG(CAST(b.gas_used AS DOUBLE) / CAST(b.gas_limit AS DOUBLE) * 100) AS avg_utilization_pct,
    AVG(CAST(t.gas_price AS DOUBLE) / 1e9) AS avg_gas_price_gwei
FROM eth.blocks b
LEFT JOIN eth.transactions t 
    ON b.number = t.block_number 
    AND b.date = t.date
WHERE b.date >= DATE_FORMAT(CURRENT_DATE - INTERVAL '7' DAY, '%Y-%m-%d')
  AND block_timestamp IS NOT NULL
GROUP BY HOUR(block_timestamp) -- some rows in your dataset have a NULL or invalid block_timestamp.
ORDER BY hour_of_day;

'''
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------
download_and_load_query_results(athena_client, response, download=False)        

# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()
if confirm == "y":
    network_congestion_patterns_last7days = download_and_load_query_results(athena_client, response,  filename="network_congestion_patterns_last7days", download=True)  
    display(network_congestion_patterns_last7days.head())
logger.info(f"Data fetched in {time.time() - start_time:.2f}s")

[2025-10-10 20:45:03,395] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:45:06,779] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 69.81 MB
[2025-10-10 20:45:06,781] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/3ebd7c03-b963-41ed-b073-073f89170e35.csv
[2025-10-10 20:49:45,324] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:49:46,400] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as network_congestion_patterns_last7days.csv
[2025-10-10 20:49:46,811] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/network_congestion_patterns_last7days.csv
[2025-10-10 20:49:46,812] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 24


,hour_of_day,total_blocks,total_transactions,avg_utilization_pct,avg_gas_price_gwei
0,0,1793,73479215,54.910632,0.954100
1,1,1789,80796247,55.933074,0.922048
2,2,1780,78440691,56.531039,0.927023
3,3,1786,69309014,56.193844,0.869733
4,4,1787,60488766,55.765772,0.939069


[2025-10-10 20:49:46,848] p21995 {795599955.py:39} INFO - Data fetched in 283.79s


# 10. Cross-chain analysis - Compare contract interactions with actual transfers

In [48]:
query = '''
-- Smart Contracts with Most Interaction (Logs + Transactions)
SELECT 
    c.address as contract_address,
    COUNT(DISTINCT t.hash) as tx_count,
    COUNT(DISTINCT l.transaction_hash) as log_event_count,
    SUM(CAST(t.value AS DOUBLE)) / 1e18 as total_eth_received,
    COUNT(DISTINCT t.from_address) as unique_users,
    c.block_timestamp as deployed_at
FROM eth.contracts c
LEFT JOIN eth.transactions t 
    ON c.address = t.to_address 
    AND c.date = t.date
LEFT JOIN eth.logs l 
    ON c.address = l.address 
    AND c.date = l.date
WHERE c.date >= DATE_FORMAT(CURRENT_DATE - INTERVAL '30' DAY, '%Y-%m-%d')
GROUP BY c.address, c.block_timestamp
HAVING COUNT(DISTINCT t.hash) > 10
ORDER BY tx_count DESC
LIMIT 20;
'''
start_time = time.time()
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE_NAME},
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT,        
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
# ---------------------------------------
# Step 1: Preview results (no download)
# ---------------------------------------
download_and_load_query_results(athena_client, response, download=False)    

# ---------------------------------------
# Step 2: Ask whether to download
# ---------------------------------------
confirm = input("Download results? (y/n): ").lower()
if confirm == "y":  
    top_contracts_by_interaction_last30days = download_and_load_query_results(athena_client, response,  filename="top_contracts_by_interaction_last30days", download=True)  
    display(top_contracts_by_interaction_last30days.head())
logger.info(f"Data fetched in {time.time() - start_time:.2f}s")



[2025-10-10 20:51:23,861] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:57:26,808] p21995 {3992468325.py:52} INFO - ✅ Query succeeded. Data scanned: 14196.07 MB
[2025-10-10 20:57:26,817] p21995 {3992468325.py:53} INFO - Output location: s3://athena-erika/data/results/1f09866c-94eb-42d4-ab09-b2435c440153.csv
[2025-10-10 20:57:29,840] p21995 {3992468325.py:34} INFO - ⏳ Waiting for query to finish...
[2025-10-10 20:57:30,280] p21995 {3992468325.py:64} INFO - 📁 Saved query result locally as top_contracts_by_interaction_last30days.csv
[2025-10-10 20:57:30,408] p21995 {3992468325.py:68} INFO - Uploaded to s3://athena-erika/data/results/top_contracts_by_interaction_last30days.csv
[2025-10-10 20:57:30,410] p21995 {3992468325.py:70} INFO - ✅ Query completed. Rows: 20


,contract_address,tx_count,log_event_count,total_eth_received,unique_users,deployed_at
0,0x28be6260dbb7872de5effc9a4a73c85b7405a8c0,32846,32275,0.000000,32148,2025-09-29 13:08:11.000
1,0x61649cd948e67e15dc76232f751f1f74bf8322a7,15168,13528,38382.118517,1,2025-09-22 15:32:11.000
2,0x0caef4fe947a5767335cce1a64502aefda5a4366,5808,5910,6905.369506,3054,2025-09-23 06:59:35.000
3,0x1178c6a8e2600f0e9bafbc5e5677fe10805f05e3,5183,5085,0.000000,5074,2025-09-29 04:32:47.000
4,0x9db8864888c700779f0131bba8fe03e74ac936c9,2024,3,198.000000,11,2025-09-18 09:14:11.000


[2025-10-10 20:57:30,447] p21995 {2446849489.py:44} INFO - Data fetched in 366.94s
